In [1]:
import os
import re
import sklearn
import regex
import emoji
import preprocessor as p
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.model_selection import GridSearchCV
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from joblib import dump, load
from sklearn.cluster import KMeans
import numpy as np

In [2]:
def lemmatize_stemming(text):
    '''
    '''
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def split_count(text):
    '''
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list


def find_hash(text):
    '''
    '''
    hashtag_list = []
    data = regex.findall(r'#\w*', text)
    for word in data:
        hashtag_list.append(word)
    return hashtag_list


def find_mention(text):
    '''
    '''
    mention_list = []
    data = regex.findall(r'@\w*', text)
    for word in data:
        mention_list.append(word)
    return mention_list


def my_preprocess(text):
    '''
    '''
    doc_emoji = split_count(text)
    doc_hash = find_hash(text)
    doc_mentions = find_mention(text)
    text = text.lower()
    text = text.replace('\\n',' ')
    text = p.clean(text)
    text = text.translate(str.maketrans(' ', ' ', punct_str))
    text = re.sub(r' \d+ ', ' ', text)
    text = re.sub(r' \d+ ', ' ', text)
    words = []
    for word in text.split(' '):
        words.append(word)
    words = [w for w in words if not w in stop_words]
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words and len(token) > 3:
            result.append(lemmatize_stemming(token))

    result = result + doc_emoji + doc_hash + doc_mentions
    tweet_txt = ' '.join(result)
    return tweet_txt


def tweet_split_2_process(tweets_str):
    '''
    '''
    output = []
    tweet_list = tweets_str.split(', ')
    for tweet in tweet_list:
        tweet_p = my_preprocess(tweet)
        output.append(tweet_p)
    
    return output        

In [40]:
def print_top_words(model, feature_names, n_top_words, file_name):
    file = open(file_name,'w') 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic%d, " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        file.write(message + '\n')
        print(message)
    file.close()
    print()

In [4]:
n_samples = 2000
n_features = 1000
n_components = 30
n_top_words = 5

stop_words = STOPWORDS.union(set(['', 'ive', 'im', 'amp', 'like', 'fuck', 'shit']))

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.EMOJI)
stemmer = SnowballStemmer('english')
punct_str = '''!"$%&'()*+,-./:;<=>?[\]^_`{|}~’'''

data_dir = '/home/ed/github/pod_tweets/follower_twts/'
# data_dir = '/run/media/ed/SD/follower_twts/'
resource_dir = '/home/ed/github/pod_tweets/resources/'

In [5]:
categories = os.listdir(data_dir)

print('Loading dataset...', end='')
docs_to_train = sklearn.datasets.load_files(
    data_dir, description=None, categories=categories, load_content=True,
    encoding='utf-8', shuffle=True, random_state=42)
print('Done')


Loading dataset...Done


In [6]:
# print(docs_to_train.target_names)

In [7]:
tweet_txt = docs_to_train['data']

In [8]:
test_txt = tweet_txt[6]
tweet_split_2_process(test_txt)
# test_list = test_txt.split(', ')
# test_list

['retweet help tweet olay donat #MAKESPACEFORWOMEN #MakeSpaceForWomen',
 '',
 'help teach girl code @GirlsWhoCode',
 'shell chang world',
 'player sport easi iverson champ helton @DrewCreasman',
 'upper brookvill work site accid trap',
 'rescu',
 'photo @LIFireBuff',
 'permiss articl site credit photograph',
 'long overdu welcom chang out @DNVR_Rockies',
 'remov free one game improv product longer watch pitcher stand look goofi plate give meaningless dont worri marquez get',
 'check late scene talk mayb quiet nolan want money need ring blame doesnt think thatll happen @purpledinocast',
 'spend instal live camera fee home plate centerfield dugout hallway need standard aluminum trash drum stick @DNVR_Rockies',
 'thank mind follow shoot @airtrain11 @News12LI @FiOS1News_LI @Newsday',
 'crazi photo @airtrain11 @News12LI @FiOS1News_LI @Newsday',
 'articl',
 'scan twitter long island news learn local play japan today start watch facebook broadcast @TylerMaun',
 'familiar voic come japan ear d

In [9]:
# test_txt_p = list(map(reformat, test_txt))

In [10]:
reformat = lambda x: tweet_split_2_process(x)
tweet_txt_p = list(map(reformat, tweet_txt))
tweet_p_list = []
for sublist in tweet_txt_p:
    tweet_p_list.extend(sublist)
# tweet_p_list[:30]

KeyboardInterrupt: 

In [7]:
resources_dir = '/home/ed/github/pod_tweets/resources/'
file_dump_name = '202002102056_tweet_p_list.joblib'

load_file = resources_dir + file_dump_name
tweet_p_list = load(load_file)

In [8]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    token_pattern=r'[^\s]+', encoding='unicode', lowercase=None, strip_accents=None, stop_words=None,
    max_df=0.95, min_df=2, max_features=n_features)
# tf_vectorizer = CountVectorizer(
#     max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(tweet_p_list)

Extracting tf features for LDA...


In [10]:
lda = LatentDirichletAllocation(
    n_components=n_components, max_iter=5, learning_method='online',
    learning_offset=50., random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=30, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [9]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

NameError: name 'lda' is not defined

In [14]:
print("Log Likelihood: ", lda.score(tf))

Log Likelihood:  -18310887.099417277


In [1]:
# # Define Search Param
# search_params = {'n_components': [10, 30], 'learning_decay': [0.5, 0.9]}

# # Init the Model
# lda = LatentDirichletAllocation(
#     n_components=n_components, max_iter=5, learning_method='online',
#     learning_offset=50., random_state=0)

# # Init Grid Search Class
# model = GridSearchCV(lda, param_grid=search_params)

# # Do the Grid Search
# model.fit(tf)

In [11]:
resources_dir = '/home/ed/github/pod_tweets/resources/'
model_dump_name = '202002110724_LDA_Grid.joblib'

load_model = resources_dir + model_dump_name
model = load(load_model)

In [12]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
# print("Model Perplexity: ", best_lda_model.perplexity(tf))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 40}
Best Log Likelihood Score:  -12353642.845903385


In [38]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(tf)
lda_output

array([[0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       ...,
       [0.55      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
        0.05      ],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
        0.05      ],
       [0.01666667, 0.01666667, 0.01666667, ..., 0.18333333, 0.1832925 ,
        0.35004084]])

In [41]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(best_lda_model, tf_feature_names, 10, resources_dir + 'topics.txt')

Topic0, go life hour kill import matter complet drive bless normal
Topic1, team bring guy what includ answer choos unit cours ❤
Topic2, time best twitter open save perform space featur visit blue
Topic3, hard problem liter class candid end light brother throw award
Topic4, today follow 🤣 amaz stay celebr yeah 🚨 test truli
Topic5, sure fight wont forget allow announc soon speech wit manag
Topic6, think talk happen doesnt american money histori mayb moment hand
Topic7, want start tri excit hold nice ticket catch friday sell
Topic8, support friend super bowl proud close shes rest wear pictur
Topic9, care write question beauti polit elect wonder human health awesom
Topic10, help week countri girl turn govern protect latest teach abus
Topic11, your read chanc meet pretti woman list havent date child
Topic12, good post true wish youv articl program luck claim 🙄
Topic13, final send readi player film perfect entir present seri arent
Topic14, trump great presid state let tonight real report par

In [14]:
# column names
topic_names = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
topic_matrix = pd.DataFrame(columns=topic_names)
topic_matrix

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39


In [1]:
categories_list = docs_to_train.target_names

for x1 in range(len(categories_list)):
    subdocs_to_train = sklearn.datasets.load_files(
        data_dir, description=None, categories=categories_list[x1], load_content=True,
        encoding='utf-8', shuffle=True, random_state=42)
    
    subset_tweet_txt = subdocs_to_train['data']
    
    print("Extracting tf features for LDA...")
    sub_tf_vectorizer = CountVectorizer(
        token_pattern=r'[^\s]+', encoding='unicode', lowercase=None, strip_accents=None, stop_words=None,
        max_features=n_features)

    reformat = lambda x: tweet_split_2_process(x)
    subset_tweet_txt_p = list(map(reformat, subset_tweet_txt))
    subset_tweet_p_list = []
    
    for sublist in subset_tweet_txt_p:
        subset_tweet_p_list.extend(sublist)

    corp_to_tf = [' '.join(subset_tweet_p_list)]

    sub_tf = sub_tf_vectorizer.fit_transform(corp_to_tf)

    X = best_lda_model.transform(sub_tf)
    
    topic_matrix.loc[categories_list[x1]] = X[0]
    
#     print(categories_list[x1], X[0])
topic_matrix

NameError: name 'docs_to_train' is not defined

In [5]:
# df = pd.DataFrame(topic_matrix.iloc[0])
# df = df.sort_values(by=['1A'], ascending=False)
# df
# print(df.columns[0])
# rank_matrix = pd.DataFrame({'1A': df.index.tolist()})
# rank_matrix
# df2 = pd.DataFrame(topic_matrix.iloc[1])
# df2 = df2.sort_values(by=['1YearDailyAudioBible'], ascending=False)
# print(df2)
# rank_matrix['1YearDailyAudioBible'] = df2.index.tolist()
# rank_matrix

rank_matrix = pd.DataFrame()
for x2 in range(len(topic_matrix)):
    df = pd.DataFrame(topic_matrix.iloc[x2])
    df = df.sort_values(by=[str(df.columns[0])], ascending=False)
    rank_matrix[df.columns[0]] = df.index.tolist()

rank_matrix.to_pickle(resources_dir + 'rank_matrix.pkl')
rank_matrix

NameError: name 'topic_matrix' is not defined

In [83]:
rank_matrix[['2DopeQueens', '1A']][:3]

,2DopeQueens,1A
0,Topic1,Topic5
1,Topic7,Topic7
2,Topic8,Topic4


In [31]:
type(X[0])

numpy.ndarray

In [43]:
print("Extracting tf features for LDA...")
sub_tf_vectorizer = CountVectorizer(
    token_pattern=r'[^\s]+', encoding='unicode', lowercase=None, strip_accents=None, stop_words=None,
    max_features=n_features)

reformat = lambda x: tweet_split_2_process(x)
subset_tweet_txt_p = list(map(reformat, subset_tweet_txt))
subset_tweet_p_list = []
for sublist in subset_tweet_txt_p:
    subset_tweet_p_list.extend(sublist)
# tweet_p_list[:30]
corp_to_tf = [' '.join(subset_tweet_p_list)]

sub_tf = sub_tf_vectorizer.fit_transform(corp_to_tf)

X = best_lda_model.transform(sub_tf)

Extracting tf features for LDA...


In [41]:
print(X.shape)
X

(1, 10)


array([[0.08205975, 0.0916345 , 0.08158102, 0.12933065, 0.09578357,
        0.12906469, 0.10913503, 0.09938296, 0.09353762, 0.0884902 ]])

In [44]:
print(X.shape)
X

(1, 10)


array([[0.08901046, 0.08111908, 0.1128255 , 0.1271588 , 0.07807929,
        0.12657501, 0.12427001, 0.067128  , 0.10016305, 0.0936708 ]])

In [88]:
# clustering topics:
clust = topic_matrix

kmeans = KMeans(n_clusters=3, random_state=0).fit(clust)

print(kmeans.labels_)

[2 1 0 0 2 0 0 2 2 2 2 0 2 1 2 2 2 1 0]


In [30]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(topic_matrix)
cos_sim_df = pd.DataFrame(cos_sim, columns=rank_matrix.columns)
cos_sim_df = cos_sim_df.T
cos_sim_df.columns = cos_sim_df.index.tolist()
# cs2 = cos_sim_df[[0]].sort_values(by=0, ascending=False)
cos_sim_df

cos_sim_pkl = 'rank_matrix_20200213.pkl'
cos_sim_df.to_pickle(resource_dir + cos_sim_pkl)
sim_list = cos_sim_df['2DopeQueens'].sort_values(ascending=False).index.tolist()
print(sim_list[1:4])
cos_sim_df['2DopeQueens'].sort_values(ascending=False)

['BehindtheBastards', 'Cults', 'CommonSensewithDanCarlin']


2DopeQueens                      1.000000
BehindtheBastards                0.976063
Cults                            0.975216
CommonSensewithDanCarlin         0.974456
ID10TwithChrisHardwick           0.973993
                                   ...   
Heavyweight                      0.945706
30For30Podcasts                  0.943290
CrimeJunkie                      0.938307
1YearDailyAudioBible             0.936216
CongratulationswithChrisDElia    0.935611
Name: 2DopeQueens, Length: 71, dtype: float64

In [17]:
topic_pkl = 'topic_matrix_202002121312.pkl'
rank_pkl = 'rank_matrix_202002121312.pkl'

topic_matrix = pd.read_pickle(resource_dir + topic_pkl)
rank_matrix = pd.read_pickle(resource_dir + rank_pkl)
pod_space_df = pd.read_pickle(resource_dir + 'pod_space_df.pkl')

In [31]:
topic_matrix

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39
1A,0.023775,0.015936,0.019031,0.026892,0.025338,0.023943,0.020521,0.015994,0.021691,0.062512,...,0.028908,0.035279,0.021136,0.017453,0.019594,0.041981,0.018098,0.015657,0.016183,0.021076
1YearDailyAudioBible,0.020417,0.017696,0.022839,0.022477,0.019558,0.018944,0.017786,0.016699,0.031636,0.045825,...,0.032210,0.036146,0.028657,0.024973,0.024131,0.035260,0.019678,0.046825,0.016941,0.017081
2DopeQueens,0.013787,0.028685,0.026791,0.030955,0.025503,0.027042,0.038922,0.016240,0.022221,0.059006,...,0.024818,0.036515,0.017318,0.013670,0.018721,0.024506,0.027720,0.016796,0.016057,0.028615
30For30Podcasts,0.023856,0.019246,0.012158,0.020779,0.013373,0.030202,0.018854,0.013391,0.033234,0.050553,...,0.032469,0.053320,0.027691,0.010358,0.023120,0.028096,0.020653,0.014147,0.017270,0.021418
48Hours,0.016950,0.039820,0.016736,0.022735,0.015799,0.018164,0.020040,0.026697,0.026883,0.064029,...,0.026452,0.042935,0.020792,0.012631,0.029549,0.027474,0.020183,0.012580,0.018654,0.029916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.021820,0.016330,0.016251,0.022492,0.028646,0.020428,0.024247,0.026154,0.024002,0.054137,...,0.015104,0.049138,0.013917,0.015227,0.024271,0.029231,0.014157,0.030782,0.030214,0.026658
LadyGang,0.017468,0.022668,0.012906,0.023661,0.026352,0.019140,0.012856,0.013328,0.025483,0.049184,...,0.020127,0.037495,0.017233,0.017402,0.014003,0.035319,0.016171,0.023897,0.016424,0.025880
PlanetMoney,0.028526,0.018198,0.014315,0.028363,0.018213,0.020493,0.028176,0.017902,0.034134,0.052870,...,0.017772,0.050168,0.023480,0.018784,0.023457,0.027345,0.015520,0.024074,0.020029,0.021313
PodSaveAmerica,0.023888,0.019253,0.015674,0.023887,0.018205,0.023500,0.018672,0.023085,0.026075,0.044770,...,0.019365,0.039474,0.016571,0.014741,0.022546,0.024704,0.020262,0.025039,0.025207,0.026746


In [32]:
pod_space_df

,x,y,pod_name
0,0.552553,-0.185661,1A
1,0.677453,-0.240002,1YearDailyAudioBible
2,0.266890,-0.034362,2DopeQueens
3,0.398905,-0.057142,30For30Podcasts
4,0.823851,-0.311452,48Hours
...,...,...,...
66,0.104882,-0.006575,KnowledgeFight
67,0.130969,0.291091,LadyGang
68,0.075300,0.060325,PlanetMoney
69,0.106649,0.195069,PodSaveAmerica


In [37]:
final_mat = topic_matrix
final_mat = final_mat.merge(pod_space_df, how='left', left_index=True, right_on='pod_name')
final_mat = final_mat.set_index('pod_name')
final_mat

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,x,y
pod_name,,,,,,,,,,,,,,,,,,,,,
1A,0.023775,0.015936,0.019031,0.026892,0.025338,0.023943,0.020521,0.015994,0.021691,0.062512,...,0.021136,0.017453,0.019594,0.041981,0.018098,0.015657,0.016183,0.021076,0.552553,-0.185661
1YearDailyAudioBible,0.020417,0.017696,0.022839,0.022477,0.019558,0.018944,0.017786,0.016699,0.031636,0.045825,...,0.028657,0.024973,0.024131,0.035260,0.019678,0.046825,0.016941,0.017081,0.677453,-0.240002
2DopeQueens,0.013787,0.028685,0.026791,0.030955,0.025503,0.027042,0.038922,0.016240,0.022221,0.059006,...,0.017318,0.013670,0.018721,0.024506,0.027720,0.016796,0.016057,0.028615,0.266890,-0.034362
30For30Podcasts,0.023856,0.019246,0.012158,0.020779,0.013373,0.030202,0.018854,0.013391,0.033234,0.050553,...,0.027691,0.010358,0.023120,0.028096,0.020653,0.014147,0.017270,0.021418,0.398905,-0.057142
48Hours,0.016950,0.039820,0.016736,0.022735,0.015799,0.018164,0.020040,0.026697,0.026883,0.064029,...,0.020792,0.012631,0.029549,0.027474,0.020183,0.012580,0.018654,0.029916,0.823851,-0.311452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.021820,0.016330,0.016251,0.022492,0.028646,0.020428,0.024247,0.026154,0.024002,0.054137,...,0.013917,0.015227,0.024271,0.029231,0.014157,0.030782,0.030214,0.026658,0.104882,-0.006575
LadyGang,0.017468,0.022668,0.012906,0.023661,0.026352,0.019140,0.012856,0.013328,0.025483,0.049184,...,0.017233,0.017402,0.014003,0.035319,0.016171,0.023897,0.016424,0.025880,0.130969,0.291091
PlanetMoney,0.028526,0.018198,0.014315,0.028363,0.018213,0.020493,0.028176,0.017902,0.034134,0.052870,...,0.023480,0.018784,0.023457,0.027345,0.015520,0.024074,0.020029,0.021313,0.075300,0.060325


In [39]:
final_mat.to_pickle(resource_dir + 'final_matrix.pkl')

In [38]:
clust = final_mat

kmeans = KMeans(n_clusters=5, random_state=0).fit(clust)

print(kmeans.labels_)

[1 1 4 4 1 2 4 2 0 4 2 0 3 3 2 2 2 2 2 0 0 2 0 2 4 2 0 2 2 2 2 0 2 2 4 2 4
 0 2 2 4 4 4 0 0 0 0 4 2 0 0 4 4 2 2 3 2 2 0 4 2 2 2 2 2 4 2 0 2 0 0]


In [9]:
rank_matrix

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,JalenJacoby,JennaJulienPodcast,JockoPodcast,JudgeJohnHodgman,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
0,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,...,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14,Topic14
1,Topic9,Topic37,Topic9,Topic20,Topic9,Topic31,Topic9,Topic9,Topic9,Topic9,...,Topic9,Topic9,Topic31,Topic9,Topic9,Topic9,Topic9,Topic9,Topic9,Topic9
2,Topic22,Topic9,Topic6,Topic31,Topic31,Topic9,Topic23,Topic31,Topic31,Topic31,...,Topic19,Topic31,Topic9,Topic34,Topic20,Topic31,Topic22,Topic31,Topic22,Topic11
3,Topic35,Topic31,Topic31,Topic9,Topic1,Topic0,Topic35,Topic30,Topic3,Topic35,...,Topic8,Topic3,Topic30,Topic31,Topic30,Topic20,Topic31,Topic24,Topic31,Topic31
4,Topic31,Topic35,Topic22,Topic8,Topic24,Topic19,Topic31,Topic24,Topic20,Topic20,...,Topic31,Topic35,Topic38,Topic35,Topic5,Topic24,Topic20,Topic17,Topic25,Topic23
5,Topic11,Topic10,Topic17,Topic30,Topic39,Topic22,Topic3,Topic25,Topic35,Topic3,...,Topic23,Topic4,Topic39,Topic20,Topic6,Topic19,Topic35,Topic8,Topic13,Topic30
6,Topic30,Topic20,Topic3,Topic5,Topic34,Topic13,Topic27,Topic11,Topic19,Topic32,...,Topic4,Topic39,Topic0,Topic11,Topic16,Topic25,Topic17,Topic0,Topic20,Topic3
7,Topic24,Topic30,Topic1,Topic22,Topic22,Topic3,Topic5,Topic22,Topic22,Topic25,...,Topic27,Topic20,Topic20,Topic8,Topic8,Topic37,Topic16,Topic20,Topic12,Topic20
8,Topic17,Topic8,Topic39,Topic35,Topic17,Topic5,Topic21,Topic37,Topic0,Topic34,...,Topic16,Topic21,Topic8,Topic24,Topic22,Topic38,Topic19,Topic3,Topic39,Topic8
9,Topic25,Topic17,Topic19,Topic32,Topic25,Topic24,Topic4,Topic35,Topic8,Topic7,...,Topic3,Topic24,Topic35,Topic17,Topic24,Topic35,Topic24,Topic19,Topic8,Topic35
